# The following are analysis performed for this Bachelor Thesis

In [ ]:
%load_ext autoreload
%autoreload 2

from tabulate import tabulate
from texttable import Texttable
import latextable
import header as head
import json
import os

annotations_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations"
community_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection"
bridges_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges"
figure_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Figures"

There are 74 rumour source tweet with 0 non-rumour source tweet within the total of 74 rumour tweet.

In [ ]:
# p = head.getPath()
# print(p)
head.count()

In [ ]:
# merging following relationship into a JSON file
head.followRelate()

with open('/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json') as f:
    data = json.load(f)
    followRelate_Ids = list(data.keys())

In [ ]:
# generate dictionary for tweetId and userId conversion
# results are saved in Id-conversions folder
head.gen_idConversion()

There are followers who reacted to the source tweets. 
* reacts from followers are saved in `react_follows`
* reacts from non-followers are saved in `react_not_follows`

In [ ]:
# See whether there are any reactors that follows the person they responded to 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json") as f:
    followingRelate = json.load(f)

head.is_follower_react(followingRelate)

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/with_bridges.json") as file:
    communities = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    who_follows_whom = json.load(file)

bridges = communities["bridges"]
rumours = communities["rumours"]
non_rumours = communities["non_rumours"]

bridges_follow = {}

for bridge in bridges:
    follows = who_follows_whom[bridge]
    bridges_follow[bridge] = {
        "rumours":[],
        "non_rumours":[],
        "bridges":[]
    }

    for f in follows:
        if f in rumours: (bridges_follow[bridge])["rumours"].append(f)
        elif f in non_rumours: (bridges_follow[bridge])["non_rumours"].append(f)
        elif f in bridges: (bridges_follow[bridge])["bridges"].append(f)

head.writeToJSON(head.getRoot(),"bridges_follows",bridges_follow)


### Reacting Relationship

In [ ]:
reacted_by = head.who_reacted_by_whom()

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as file:
    communities = json.load(file)

rumours = communities["rumours"]
non_rumours = communities["non_rumours"]
uncategorized = communities["uncategorized"]

cat_who_reacted_by_whom = {}

for id in reacted_by:
    if not(id in rumours or id in non_rumours or id in uncategorized):
        continue
    cat_who_reacted_by_whom[id] = {}
    user = cat_who_reacted_by_whom[id]
    user["rumours"] = list(set(reacted_by[id]).intersection(set(rumours)))
    user["non_rumours"] = list(set(reacted_by[id]).intersection(set(non_rumours)))
    user["uncategorized"] = list(set(reacted_by[id]).intersection(set(uncategorized)))

    # pop keys with no reactants
    if not(user["rumours"] or user["non_rumours"] or user["uncategorized"]):
        cat_who_reacted_by_whom.pop(id)

head.writeToJSON(head.getRoot(),"who_reacted_by_whom",cat_who_reacted_by_whom)

In [ ]:
# Filter out type1 bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/react/type1.json") as file:
    react_type1 = json.load(file)
    react_type1 = react_type1.keys()

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_reacted_by_whom.json") as file:
    who_follows_whom = json.load(file)

for key, values in who_follows_whom.items():
    bridges = []
    for id in values["rumours"]:
        if id in react_type1:
            bridges.append(id)
            values["rumours"].remove(id)
    
    for id in values["non_rumours"]:
        if id in react_type1:
                bridges.append(id)
                values["non_rumours"].remove(id)
    
    for id in values["uncategorized"]:
        if id in react_type1:
            bridges.append(id)
            values["uncategorized"].remove(id)
    
    (who_follows_whom[key])["bridges"] = bridges

head.writeToJSON(head.getRoot(),"who_reacted_by_whom",who_follows_whom)


Merge following and reacting relationships into one

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_reacted_by_whom.json") as file:
    reacting_relate = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as file:
    following_relate = json.load(file)

react_ids = reacting_relate.keys()
follow_id = following_relate.keys()

duplicates_ids = list(set(react_ids).intersection(set(follow_id)))
# react_ids = list(set(react_ids).difference(set(duplicates_ids)))
follow_id = follow_id - duplicates_ids

for key, values in reacting_relate.items():
    try:
        values["rumours"] += list(set((following_relate[key])["rumours"]) - set( values["rumours"]))
        values["non_rumours"] += list(set((following_relate[key])["non_rumours"]) - set( values["non_rumours"]))
        values["bridges"] += list(set((following_relate[key])["bridges"]) - set( values["bridges"]))
        values["uncategorized"] += list(set((following_relate[key])["uncategorized"]) - set( values["uncategorized"]))
    except:
        continue

for follow in follow_id:
    reacting_relate[follow] = following_relate[follow]

head.writeToJSON(head.getRoot(),"react+following",reacting_relate)

### Tabular

In [ ]:
# follow + follower counts
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/centrality.json") as file:
    bridge_centrality = json.load(file)

table = Texttable()
table.set_cols_align(["c", "c", "c"])
table.set_cols_valign(["t", "m", "b"])
table.add_rows([["id","#rumour followers","#non-rumour followers"]])

data = []
for id in bridge_centrality:
    bridge = bridge_centrality[id]
    info = [id,bridge["rum_count"],bridge["nonrum_count"]]
    data.append(info)

table.add_rows(data)
with open('follower_counts.txt', 'w') as f:
    f.write(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))



In [ ]:
# follow + betweeness + centrality
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/centrality_2.json") as file:
    bridge_centrality = json.load(file)

table = Texttable()
table.set_cols_align(["c", "c", "c"])
table.set_cols_valign(["t", "t", "t"])
table.set_cols_dtype(["i","e","e"])

data = []
for id in bridge_centrality:
    bridge = bridge_centrality[id]
    info = []
    if bridge["rumours"]:
        info.append([id,"betweeness","centrality"])
        info.append(["rumours","",""])
        
        for follower in bridge["rumours"]:
            user = (bridge["rumours"])[follower]
            info.append([follower,user["betweeness"],user["closeness"]])
    
    elif bridge["non_rumours"]:
        info.append([id,"betweeness","centrality"])
        info.append(["non-rumours","",""])

        for follower in bridge["non_rumours"]:
            user = (bridge["non_rumours"])[follower]
            info.append([follower,user["betweeness"],user["closeness"]])
    
    data.extend(info)

table.add_rows(data)
# print(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))
with open(head.makePath([figure_path,"follow","follower_centrality.txt"]), 'w') as f:
    f.write(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))

